# Week 7 -- Data Cleaning

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings
warnings.simplefilter('ignore')

In [66]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

## Functions

In [67]:
def check_names(df1, df2):
    """
    This funciton compares the list of names from one dataframe to another and updates where the names do not match 
    so they will merge correctly.
    
    Parameters:
        df1: first dataframe to compare to; this should be the dataframe on the left that you will join with when 
        they merge.
        df2: second dataframe, should be the dataframe on the right for the later merge.
        
    Returns:
        Updated name list for df2 that should match df1.
    """
    
    df_players = list(df1['Player'])    
    for i in range(len(df2['Player'])):
        name = df2['Player'][i]
        split_name = name.split(' ')
        if (split_name[0] + ' ' + split_name[1]) in df_players:
            df2['Player'][i] = split_name[0] + ' ' + split_name[1]
        else:
            df2['Player'][i] = name

In [68]:
def name_update(dataframe, column):
    """
    This function updates a column in a dataframe by stripping any excess spaces surrounding the observation.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column: the column you want to update.
    
    Returns:
        Updated dataframe column.    
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip())

In [69]:
def update_int_dtype(dataframe, column_list):
    """
    This function updates the data type for a list of columns to integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column data types to integer.   
    """
    for column in column_list:
        dataframe[column] = dataframe[column].map(lambda x: int(x))

In [70]:
def update_float_dtype(dataframe, column_list):
    """
    This function takes a dataframe object and converts it to a float.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
    
    Returns:
        Updated dataframe column data types to a float.  
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = '0.0'
            else:
                dataframe[column][i] = item 
            updated_item = str(dataframe[column][i])
            ones, tenths = updated_item.split('.')
            ones = int(ones)
            tenths = int(tenths) * .1
            dataframe[column][i] = ones + tenths
        dataframe[column] = dataframe[column].map(lambda x: float(x))

In [71]:
def fill_blanks(dataframe, column_list):
    """
    This function fills blank values with 0. This should be used for a column that will be an integer.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.    
    
    Returns:
        Updated dataframe columns with filled values.      
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == '':
                dataframe[column][i] = 0
            else:
                dataframe[column][i] = item

In [72]:
def remove_comma(dataframe, column_list):
    """
    This function removes the comma from a value in a column.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns less any commas that might appear.    
    """  
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item.split(',')) > 1:
                one, two = item.split(',')
                dataframe[column][i] = one + two

In [73]:
def adjust_float(dataframe, column_list):
    """
    This function adjusts values in a column to be workable with the 'update_float_dtype' function.
    
    Parameters:
        dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe columns with objects with a decimal point value behind it.    
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if len(item) < 3:
                dataframe[column][i] = item + '.0'
            else:
                dataframe[column][i] = item

In [74]:
def update_percent(dataframe, column):
    """
    This function updates a column with a percent sign so it can be manipulated into a float with the 
    'update_float_dtype' function.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed.     
    """
    dataframe[column] = dataframe[column].map(lambda x: x.strip('%'))
    for i in range(len(dataframe[column])):
        item = dataframe[column][i]
        if len(item) < 3:
            dataframe[column][i] = item + '.0'
        else:
            dataframe[column][i] = item

In [75]:
def replace_dash(dataframe, column_list, dash):
    """
    This function replaces a - in a dataframe so datatypes can then be updated.
    
    Parameters:
       dataframe: the dataframe you want to update.
        column_list: list of columns to iterate through.
        dash: set to a value in a specific column in the dataframe.
        
    Returns:
        Updated dataframe column with no percent sign and added decimal point where needed. 
    """
    for column in column_list:
        for i in range(len(dataframe[column])):
            item = dataframe[column][i]
            if item == dash:
                dataframe[column][i] = '0'
            else:
                dataframe[column][i] = item

## Read in Collected Data

In [76]:
df = pd.read_pickle('player_stats')
fantasy_pts = pd.read_pickle('fantasy_weeks')
defense = pd.read_pickle('defense_data')
kicking = pd.read_pickle('kicking_data')

## Merge Last Week Fantasy Pts with this Week

In [77]:
no_games = 7

In [78]:
lastwk = pd.read_pickle('fantweeks_1_' + str(no_games - 1))
lastwk.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Kyler Murray,ARI,QB,164.4,27.3,33.1,24.7,23.1,27.3,28.9
1,Patrick Mahomes II,KC,QB,159.4,20.4,27.5,40.0,20.2,30.7,20.6
2,Josh Allen,BUF,QB,154.7,28.2,34.5,32.2,25.4,18.3,16.1
3,Russell Wilson,SEA,QB,150.4,31.8,34.4,36.8,21.9,25.5,NaN
4,Dak Prescott,DAL,QB,138.6,17.6,39.8,29.5,38.3,13.4,NaN


In [79]:
fantasy_pts = pd.merge(fantasy_pts, lastwk, 'left', on='Player')
fantasy_pts.head()

,Player,Team_x,Position_x,TTL_x,Week_7,Team_y,Position_y,TTL_y,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6
0,Kyler Murray,ARI,QB,202.5,38.1,ARI,QB,164.4,27.3,33.1,24.7,23.1,27.3,28.9
1,Russell Wilson,SEA,QB,183.3,32.9,SEA,QB,150.4,31.8,34.4,36.8,21.9,25.5,NaN
2,Patrick Mahomes II,KC,QB,171.4,12.0,KC,QB,159.4,20.4,27.5,40.0,20.2,30.7,20.6
3,Josh Allen,BUF,QB,171.1,16.4,BUF,QB,154.7,28.2,34.5,32.2,25.4,18.3,16.1
4,Deshaun Watson,HOU,QB,159.6,24.2,HOU,QB,135.4,21.8,15.7,18.1,20.9,26.9,32.0


In [80]:
fantasy_pts.drop(columns=['Team_y', 'Position_y', 'TTL_y'], inplace=True)
fantasy_pts.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position', 'TTL_x': 'TTL'}, inplace = True)
fantasy_pts.columns

Index(['Player', 'Team', 'Position', 'TTL', 'Week_7', 'Week_1', 'Week_2',
       'Week_3', 'Week_4', 'Week_5', 'Week_6'],
      dtype='object')

In [81]:
fantasy_pts = fantasy_pts[['Player', 'Team', 'Position', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 
                           'Week_6', 'Week_7']]
fantasy_pts.head()

,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Kyler Murray,ARI,QB,202.5,27.3,33.1,24.7,23.1,27.3,28.9,38.1
1,Russell Wilson,SEA,QB,183.3,31.8,34.4,36.8,21.9,25.5,NaN,32.9
2,Patrick Mahomes II,KC,QB,171.4,20.4,27.5,40.0,20.2,30.7,20.6,12.0
3,Josh Allen,BUF,QB,171.1,28.2,34.5,32.2,25.4,18.3,16.1,16.4
4,Deshaun Watson,HOU,QB,159.6,21.8,15.7,18.1,20.9,26.9,32.0,24.2


In [82]:
print(fantasy_pts.shape)

(631, 11)


### Pickle Combined DataFrame for Next Week

In [83]:
fantasy_pts.to_pickle('fantweeks_1_' + str(no_games))

## Merge Offensive Players & Weekly Fantasy Pts

In [84]:
print(df.shape)
df.head()

(531, 28)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank
0,Kyler Murray,ARI,QB,23,7,7,169,253,1847,13,7,65,437,6.72,7,0,0,0,,0,2,1,7,,,202.6,1,1
1,Alvin Kamara,NOR,RB,25,6,5,0,0,0,0,0,75,364,4.85,4,53,46,460,10.00,3,0,0,7,,,147.4,1,2
2,Derrick Henry,TEN,RB,26,6,6,0,0,0,0,0,143,663,4.64,7,17,10,81,8.10,0,0,0,7,,,121.4,2,3
3,Russell Wilson,SEA,QB,32,6,6,156,219,1890,22,6,29,237,8.17,0,0,0,0,,0,1,0,0,,1,183.3,2,4
4,Calvin Ridley,ATL,WR,26,7,7,0,0,0,0,0,3,14,4.67,0,64,40,615,15.38,6,0,0,6,1,,120.9,1,5


In [85]:
print(fantasy_pts.shape)
fantasy_pts.head()

(631, 11)


,Player,Team,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Kyler Murray,ARI,QB,202.5,27.3,33.1,24.7,23.1,27.3,28.9,38.1
1,Russell Wilson,SEA,QB,183.3,31.8,34.4,36.8,21.9,25.5,NaN,32.9
2,Patrick Mahomes II,KC,QB,171.4,20.4,27.5,40.0,20.2,30.7,20.6,12.0
3,Josh Allen,BUF,QB,171.1,28.2,34.5,32.2,25.4,18.3,16.1,16.4
4,Deshaun Watson,HOU,QB,159.6,21.8,15.7,18.1,20.9,26.9,32.0,24.2


In [86]:
check_names(df, fantasy_pts)

In [87]:
name_update(df, 'Player')
name_update(fantasy_pts, 'Player')

In [88]:
offense = pd.merge(df, fantasy_pts, 'left', on='Player')

In [89]:
offense.drop(columns = ['Team_y', 'Position_y'], inplace = True)
offense.rename(columns = {'Team_x': 'Team', 'Position_x': 'Position'}, inplace = True)

In [90]:
print(offense.shape)
offense.head(10)

(531, 36)


,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Kyler Murray,ARI,QB,23,7,7,169,253,1847,13,7,65,437,6.72,7,0,0,0,,0,2,1,7,,,202.6,1,1,202.5,27.3,33.1,24.7,23.1,27.3,28.9,38.1
1,Alvin Kamara,NOR,RB,25,6,5,0,0,0,0,0,75,364,4.85,4,53,46,460,10.00,3,0,0,7,,,147.4,1,2,124.4,18.7,29.4,31.7,17.9,11.9,NaN,14.8
2,Derrick Henry,TEN,RB,26,6,6,0,0,0,0,0,143,663,4.64,7,17,10,81,8.10,0,0,0,7,,,121.4,2,3,116.4,13.1,8.4,25.0,NaN,18.3,38.4,13.2
3,Russell Wilson,SEA,QB,32,6,6,156,219,1890,22,6,29,237,8.17,0,0,0,0,,0,1,0,0,,1,183.3,2,4,183.3,31.8,34.4,36.8,21.9,25.5,NaN,32.9
4,Calvin Ridley,ATL,WR,26,7,7,0,0,0,0,0,3,14,4.67,0,64,40,615,15.38,6,0,0,6,1,,120.9,1,5,100.9,24.9,22.9,11.7,0.0,13.6,12.9,14.9
5,James Robinson,JAX,RB,22,7,7,0,0,0,0,0,107,481,4.50,4,32,27,225,8.33,2,2,1,6,1,,120.1,3,6,106.6,9.0,18.0,24.9,10.7,5.0,11.3,27.7
6,Tyler Lockett,SEA,WR,28,6,6,0,0,0,0,0,0,0,,0,58,45,542,12.04,7,0,0,7,,,118.7,2,7,96.2,9.2,12.7,28.0,3.9,4.4,NaN,38.0
7,Patrick Mahomes,KAN,QB,25,7,7,159,242,1899,16,1,34,165,4.85,2,0,0,0,,0,1,0,2,,2,171.5,3,8,171.4,20.4,27.5,40.0,20.2,30.7,20.6,12.0
8,Travis Kelce,KAN,TE,31,7,7,0,0,0,0,0,0,0,,0,56,40,501,12.53,5,1,1,5,,,98.1,1,9,78.1,11.0,15.0,8.7,7.0,16.8,16.5,3.1
9,Dalvin Cook,MIN,RB,25,5,5,0,0,0,0,0,92,489,5.32,7,16,12,64,5.33,0,1,1,7,3,,107.3,4,10,101.3,20.8,15.1,23.9,26.6,14.9,NaN,NaN


In [91]:
offense.isnull().sum()

Player                     0
Team                       0
Position                   0
Age                        0
Games                      0
GamesStarted               0
CompletedPasses            0
PassesAttempted            0
PassingYds                 0
PassingTDs                 0
Interceptions              0
RushingAttempts            0
RushingYds                 0
RushingYdspAtt             0
RushingTDs                 0
Targeted                   0
Receptions                 0
ReceivingYds               0
YdspReception              0
ReceivingTDs               0
Fumbles                    0
LostFumbles                0
TtlTDs                     0
TwoPTConversions           0
TwoPTConversionPasses      0
FDFantasyPts               0
PositionRank               0
OverallRank                0
TTL                        9
Week_1                   124
Week_2                   126
Week_3                   127
Week_4                   153
Week_5                   182
Week_6        

In [92]:
offense.fillna('0.0', inplace = True)

In [93]:
off_integers = ['Age', 'Games', 'GamesStarted', 'CompletedPasses', 'PassesAttempted', 'PassingYds', 'PassingTDs', 
            'Interceptions', 'RushingAttempts', 'RushingYds', 'RushingTDs', 'Targeted', 'Receptions', 
            'ReceivingYds', 'ReceivingTDs', 'Fumbles', 'LostFumbles', 'TtlTDs']

off_floats = ['RushingYdspAtt', 'YdspReception', 'FDFantasyPts', 'TTL','Week_1', 'Week_2', 'Week_3', 'Week_4', 
              'Week_5', 'Week_6', 'Week_7']

In [94]:
update_int_dtype(offense, off_integers)

In [95]:
two_pts = ['TwoPTConversions', 'TwoPTConversionPasses']
fill_blanks(offense, two_pts)
update_int_dtype(offense, two_pts)

In [96]:
update_float_dtype(offense, off_floats)

In [97]:
offense.head()

,Player,Team,Position,Age,Games,GamesStarted,CompletedPasses,PassesAttempted,PassingYds,PassingTDs,Interceptions,RushingAttempts,RushingYds,RushingYdspAtt,RushingTDs,Targeted,Receptions,ReceivingYds,YdspReception,ReceivingTDs,Fumbles,LostFumbles,TtlTDs,TwoPTConversions,TwoPTConversionPasses,FDFantasyPts,PositionRank,OverallRank,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Kyler Murray,ARI,QB,23,7,7,169,253,1847,13,7,65,437,13.2,7,0,0,0,0.0,0,2,1,7,0,0,202.6,1,1,202.5,27.3,33.1,24.7,23.1,27.3,28.9,38.1
1,Alvin Kamara,NOR,RB,25,6,5,0,0,0,0,0,75,364,12.5,4,53,46,460,10.0,3,0,0,7,0,0,147.4,1,2,124.4,18.7,29.4,31.7,17.9,11.9,0.0,14.8
2,Derrick Henry,TEN,RB,26,6,6,0,0,0,0,0,143,663,10.4,7,17,10,81,9.0,0,0,0,7,0,0,121.4,2,3,116.4,13.1,8.4,25.0,0.0,18.3,38.4,13.2
3,Russell Wilson,SEA,QB,32,6,6,156,219,1890,22,6,29,237,9.7,0,0,0,0,0.0,0,1,0,0,0,1,183.3,2,4,183.3,31.8,34.4,36.8,21.9,25.5,0.0,32.9
4,Calvin Ridley,ATL,WR,26,7,7,0,0,0,0,0,3,14,10.7,0,64,40,615,18.8,6,0,0,6,1,0,120.9,1,5,100.9,24.9,22.9,11.7,0.0,13.6,12.9,14.9


In [98]:
team_dict = {'LAR': 'Los Angeles Raiders', 
             'SEA': 'Seattle Seahawks', 
             'BAL': 'Baltimore Ravens', 
             'KAN': 'Kansas City Chiefs', 
             'JAX': 'Jacksonville Jaguars', 
             'DET': 'Detroit Lions', 
             'PIT': 'Pittsburgh Steelers', 
             'MIN': 'Minnesota Vikings', 
             'CHI': 'Chicago Bears', 
             'ATL': 'Atlanta Falcons', 
             'BUF': 'Buffalo Bills', 
             'GNB': 'Green Bay Packers', 
             'MIA': 'Miami Dolphins', 
             'CLE': 'Cleveland Browns', 
             'CAR': 'Carolina Panthers', 
             'DAL': 'Dallas Cowboys', 
             'ARI': 'Arizona Cardinals', 
             'HOU': 'Houston Texans', 
             'NYG': 'New York Giants', 
             'WAS': 'Washington Football Team', 
             'IND': 'Indianapolis Colts', 
             'LVR': 'Las Vegas Raiders', 
             'DEN': 'Denver Broncos', 
             'TEN': 'Tennessee Titans', 
             'NWE': 'New England Patriots', 
             'CIN': 'Cincinnati Bengals', 
             'NYJ': 'New York Jets', 
             'LAC': 'Los Angeles Chargers', 
             'PHI': 'Philadelphia Eagles', 
             'TAM': 'Tampa Bay Buccaneers', 
             'NOR': 'New Orleans Saints', 
             'SFO': 'San Francisco 49ers'}

team_names = pd.DataFrame()
team_names['Team'] = list(team_dict.keys())
team_names['Long_Name'] = list(team_dict.values())
team_names.head()

,Team,Long_Name
0,LAR,Los Angeles Raiders
1,SEA,Seattle Seahawks
2,BAL,Baltimore Ravens
3,KAN,Kansas City Chiefs
4,JAX,Jacksonville Jaguars


In [99]:
offense.dtypes

Player                    object
Team                      object
Position                  object
Age                        int64
Games                      int64
GamesStarted               int64
CompletedPasses            int64
PassesAttempted            int64
PassingYds                 int64
PassingTDs                 int64
Interceptions              int64
RushingAttempts            int64
RushingYds                 int64
RushingYdspAtt           float64
RushingTDs                 int64
Targeted                   int64
Receptions                 int64
ReceivingYds               int64
YdspReception            float64
ReceivingTDs               int64
Fumbles                    int64
LostFumbles                int64
TtlTDs                     int64
TwoPTConversions           int64
TwoPTConversionPasses      int64
FDFantasyPts             float64
PositionRank              object
OverallRank               object
TTL                      float64
Week_1                   float64
Week_2    

## Defense Stats

In [100]:
defense.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks
0,Baltimore Ravens,6,104,373,5.1,"2,039",143,654,28.8,4,232,40,149,61.9,232,8,12,9,66%,11,3,8,22
1,Miami Dolphins,6,113,349,5.8,"2,161",149,745,30.2,8,209,46.3,130,71.4,209,7,18,15,75.6833333333333%,9,5,4,17
2,Los Angeles Rams,7,114,363,5,"1,911",150,654,25.9,5,214,38.9,139,59.1,214,8,23,15,63.9333333333333%,6,4,2,20
3,Indianapolis Colts,6,115,315,4.8,"1,728",151,530,20.9,4,194,39.7,120,63.4,194,7,13,8,61.35%,10,10,0,13
4,Chicago Bears,7,116,416,5.1,"2,023",159,678,26.7,5,219,39.6,125,70,219,4,28,10,38.95%,8,5,3,15


In [101]:
def_fantasy = fantasy_pts[fantasy_pts['Position'] == 'DST']

In [102]:
defense_df = pd.merge(defense, def_fantasy, 'left', left_on = 'Team', right_on = 'Player')

In [103]:
defense_df.head()

,Team_x,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,Player,Team_y,Position,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Baltimore Ravens,6,104,373,5.1,"2,039",143,654,28.8,4,232,40,149,61.9,232,8,12,9,66%,11,3,8,22,Baltimore Ravens,BAL,DST,70.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN
1,Miami Dolphins,6,113,349,5.8,"2,161",149,745,30.2,8,209,46.3,130,71.4,209,7,18,15,75.6833333333333%,9,5,4,17,Miami Dolphins,MIA,DST,46.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN
2,Los Angeles Rams,7,114,363,5,"1,911",150,654,25.9,5,214,38.9,139,59.1,214,8,23,15,63.9333333333333%,6,4,2,20,Los Angeles Rams,LAR,DST,53.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0
3,Indianapolis Colts,6,115,315,4.8,"1,728",151,530,20.9,4,194,39.7,120,63.4,194,7,13,8,61.35%,10,10,0,13,Indianapolis Colts,IND,DST,61.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN
4,Chicago Bears,7,116,416,5.1,"2,023",159,678,26.7,5,219,39.6,125,70,219,4,28,10,38.95%,8,5,3,15,Chicago Bears,CHI,DST,47.0,3.0,12.0,4.0,2.0,6.0,11.0,9.0


In [104]:
defense_df.drop(columns=['Player', 'Team_y', 'Position'], inplace=True)
defense_df.rename(columns={'Team_x': 'Team'}, inplace=True)

In [105]:
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Baltimore Ravens,6,104,373,5.1,"2,039",143,654,28.8,4,232,40,149,61.9,232,8,12,9,66%,11,3,8,22,70.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN
1,Miami Dolphins,6,113,349,5.8,"2,161",149,745,30.2,8,209,46.3,130,71.4,209,7,18,15,75.6833333333333%,9,5,4,17,46.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN
2,Los Angeles Rams,7,114,363,5,"1,911",150,654,25.9,5,214,38.9,139,59.1,214,8,23,15,63.9333333333333%,6,4,2,20,53.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0
3,Indianapolis Colts,6,115,315,4.8,"1,728",151,530,20.9,4,194,39.7,120,63.4,194,7,13,8,61.35%,10,10,0,13,61.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN
4,Chicago Bears,7,116,416,5.1,"2,023",159,678,26.7,5,219,39.6,125,70,219,4,28,10,38.95%,8,5,3,15,47.0,3.0,12.0,4.0,2.0,6.0,11.0,9.0


In [106]:
defense_df.isnull().sum()

Team                         0
GP                           0
Ttl_Pts_Allowed              0
Ttl_Offense_Plays_Allowed    0
Yds_p_Play                   0
Ttl_Yds                      0
Rushing_Att                  0
Rushing_Yds                  0
Rushing_Yds_p_Att            0
Rushing_TDs                  0
Passing_Att                  0
Passing_Yds_p_Att            0
Completions                  0
Yds_p_Completion             0
Passing_Yds                  0
Passing_TDs                  0
RZ_Att                       0
RZ_TD                        0
RZ_Percent                   0
Ttl_Turnovers                0
Interceptions                0
Fumbles                      0
Sacks                        0
TTL                          0
Week_1                       0
Week_2                       0
Week_3                       0
Week_4                       0
Week_5                       4
Week_6                       4
Week_7                       4
dtype: int64

In [107]:
defense_df

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Baltimore Ravens,6,104,373,5.1,"2,039",143,654,28.8,4,232,40,149,61.9,232,8,12,9,66%,11,3,8,22,70.0,15.0,15.0,1.0,6.0,26.0,7.0,NaN
1,Miami Dolphins,6,113,349,5.8,"2,161",149,745,30.2,8,209,46.3,130,71.4,209,7,18,15,75.6833333333333%,9,5,4,17,46.0,2.0,2.0,12.0,3.0,12.0,15.0,NaN
2,Los Angeles Rams,7,114,363,5,"1,911",150,654,25.9,5,214,38.9,139,59.1,214,8,23,15,63.9333333333333%,6,4,2,20,53.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0
3,Indianapolis Colts,6,115,315,4.8,"1,728",151,530,20.9,4,194,39.7,120,63.4,194,7,13,8,61.35%,10,10,0,13,61.0,4.0,15.0,26.0,7.0,5.0,4.0,NaN
4,Chicago Bears,7,116,416,5.1,"2,023",159,678,26.7,5,219,39.6,125,70,219,4,28,10,38.95%,8,5,3,15,47.0,3.0,12.0,4.0,2.0,6.0,11.0,9.0
5,Pittsburgh Steelers,6,118,362,4.9,"1,718",122,413,39.6,4,202,44.1,119,74.9,202,11,21,10,66.5333333333333%,9,8,1,26,56.0,8.0,13.0,7.0,10.0,8.0,18.0,2.0
6,San Francisco 49ers,7,136,406,5.2,"2,167",182,744,47,4,223,47.5,139,74.6,223,8,12,6,31.3571428571429%,46,8,2,12,45.0,4.0,5.0,12.0,6.0,-2.0,3.0,17.0
7,Tampa Bay Buccaneers,7,142,420,4.8,"2,039",154,462,42,5,250,50.5,168,74.6,250,9,27,13,56.1571428571428%,21,9,3,25,67.0,0.0,14.0,16.0,6.0,6.0,19.0,6.0
8,Kansas City Chiefs,7,143,479,5.5,"2,531",213,"1,049",63.2,7,228,47.4,138,77.1,228,9,24,17,80.2142857142857%,31,9,4,16,59.0,7.0,5.0,7.0,20.0,-1.0,3.0,18.0
9,New England Patriots,6,143,347,6.1,"2,162",178,793,58.4,5,168,51.7,115,79,168,9,20,13,63.8333333333333%,30,8,4,9,47.0,11.0,6.0,15.0,4.0,NaN,7.0,4.0


In [108]:
defense_df.fillna('0.0', inplace=True)

In [109]:
def_int = ['GP', 'Ttl_Pts_Allowed', 'Ttl_Offense_Plays_Allowed', 'Ttl_Yds', 'Rushing_Att', 'Rushing_Yds', 
           'Rushing_TDs', 'Passing_Att', 'Completions', 'Passing_Yds', 'Passing_TDs', 'RZ_Att', 'RZ_TD', 
           'Ttl_Turnovers', 'Interceptions', 'Fumbles', 'Sacks']

def_floats = ['Yds_p_Play', 'Rushing_Yds_p_Att', 'Passing_Yds_p_Att', 'Yds_p_Completion', 'TTL', 'Week_1', 
              'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7']

In [110]:
remove_comma(defense_df, def_int)

In [111]:
update_int_dtype(defense_df, def_int)

In [112]:
adjust_float(defense_df, def_floats)

In [113]:
update_float_dtype(defense_df, def_floats)

In [114]:
update_percent(defense_df, 'RZ_Percent')

In [115]:
rz = ['RZ_Percent']
update_float_dtype(defense_df, rz)

In [116]:
defense_df.head()

,Team,GP,Ttl_Pts_Allowed,Ttl_Offense_Plays_Allowed,Yds_p_Play,Ttl_Yds,Rushing_Att,Rushing_Yds,Rushing_Yds_p_Att,Rushing_TDs,Passing_Att,Passing_Yds_p_Att,Completions,Yds_p_Completion,Passing_Yds,Passing_TDs,RZ_Att,RZ_TD,RZ_Percent,Ttl_Turnovers,Interceptions,Fumbles,Sacks,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Baltimore Ravens,6,104,373,5.1,2039,143,654,28.8,4,232,40.0,149,61.9,232,8,12,9,66.0,11,3,8,22,70.0,15.0,15.0,1.0,6.0,26.0,7.0,0.0
1,Miami Dolphins,6,113,349,5.8,2161,149,745,30.2,8,209,46.3,130,71.4,209,7,18,15,683333333408.3,9,5,4,17,46.0,2.0,2.0,12.0,3.0,12.0,15.0,0.0
2,Los Angeles Rams,7,114,363,5.0,1911,150,654,25.9,5,214,38.9,139,59.1,214,8,23,15,933333333396.3,6,4,2,20,53.0,4.0,7.0,4.0,11.0,12.0,0.0,15.0
3,Indianapolis Colts,6,115,315,4.8,1728,151,530,20.9,4,194,39.7,120,63.4,194,7,13,8,64.5,10,10,0,13,61.0,4.0,15.0,26.0,7.0,5.0,4.0,0.0
4,Chicago Bears,7,116,416,5.1,2023,159,678,26.7,5,219,39.6,125,70.0,219,4,28,10,47.5,8,5,3,15,47.0,3.0,12.0,4.0,2.0,6.0,11.0,9.0


In [117]:
defense_df.dtypes

Team                          object
GP                             int64
Ttl_Pts_Allowed                int64
Ttl_Offense_Plays_Allowed      int64
Yds_p_Play                   float64
Ttl_Yds                        int64
Rushing_Att                    int64
Rushing_Yds                    int64
Rushing_Yds_p_Att            float64
Rushing_TDs                    int64
Passing_Att                    int64
Passing_Yds_p_Att            float64
Completions                    int64
Yds_p_Completion             float64
Passing_Yds                    int64
Passing_TDs                    int64
RZ_Att                         int64
RZ_TD                          int64
RZ_Percent                   float64
Ttl_Turnovers                  int64
Interceptions                  int64
Fumbles                        int64
Sacks                          int64
TTL                          float64
Week_1                       float64
Week_2                       float64
Week_3                       float64
W

## Kicking Stats

In [118]:
kicking.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS
0,Randy Bullock,K,CIN,7,19,89.5,55,0-0,3-3,6-7,6-7,2-2,16,100.0,67
1,Joey Slye,K,CAR,7,20,85.0,48,0-0,8-8,5-5,4-5,0-2,15,86.7,64
2,Rodrigo Blankenship,K,IND,6,18,88.9,44,0-0,4-4,7-8,5-6,0-0,15,100.0,63
3,Justin Tucker,K,BAL,6,14,92.9,55,0-0,2-2,3-3,7-7,1-2,20,100.0,59
4,Harrison Butker,K,KC,7,14,92.9,58,1-1,3-3,6-6,1-2,2-2,24,79.2,58


In [119]:
kicking_df = pd.merge(kicking, fantasy_pts, left_on = 'PLAYER', right_on = 'Player')
kicking_df.drop(columns = ['Player', 'Team', 'Position'], inplace = True)
kicking_df

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Randy Bullock,K,CIN,7,19,89.5,55,0-0,3-3,6-7,6-7,2-2,16,100.0,67,77.0,10.0,13.0,12.0,17.0,3.0,12.0,10.0
1,Joey Slye,K,CAR,7,20,85.0,48,0-0,8-8,5-5,4-5,0-2,15,86.7,64,68.0,12.0,5.0,15.0,7.0,11.0,11.0,7.0
2,Rodrigo Blankenship,K,IND,6,18,88.9,44,0-0,4-4,7-8,5-6,0-0,15,100.0,63,68.0,8.0,15.0,12.0,14.0,11.0,8.0,NaN
3,Justin Tucker,K,BAL,6,14,92.9,55,0-0,2-2,3-3,7-7,1-2,20,100.0,59,68.0,9.0,16.0,9.0,8.0,10.0,16.0,NaN
4,Harrison Butker,K,KC,7,14,92.9,58,1-1,3-3,6-6,1-2,2-2,24,79.2,58,63.0,10.0,13.0,4.0,8.0,6.0,8.0,14.0
5,Younghoe Koo,K,ATL,6,16,93.8,54,0-0,3-3,3-3,7-8,2-2,15,86.7,58,69.0,9.0,17.0,8.0,NaN,13.0,20.0,2.0
6,Jason Sanders,K,MIA,6,15,100.0,52,0-0,5-5,2-2,6-6,2-2,13,100.0,58,68.0,4.0,11.0,7.0,18.0,22.0,6.0,NaN
7,Ryan Succop,K,TB,7,13,84.6,50,0-0,3-3,5-5,2-3,1-2,26,96.2,58,62.0,5.0,7.0,9.0,8.0,14.0,10.0,9.0
8,Daniel Carlson,K,LV,6,14,92.9,54,0-0,5-5,3-3,2-3,3-3,19,94.7,57,65.0,12.0,12.0,8.0,13.0,11.0,NaN,9.0
9,Tyler Bass,K,BUF,7,17,70.6,53,1-1,2-2,3-6,5-6,1-2,19,94.7,54,61.0,9.0,7.0,5.0,6.0,5.0,6.0,23.0


In [120]:
dash = kicking_df['LNG'][39]
dash

'—'

In [122]:
kicking_df.fillna('0.0', inplace = True)

In [123]:
kick_int = ['GP', 'FGM_A', 'LNG', 'XPM_A', 'PTS']

kick_float = ['FG%', 'XP%', 'TTL', 'Week_1', 'Week_2', 'Week_3', 'Week_4', 'Week_5', 'Week_6', 'Week_7']

In [124]:
replace_dash(kicking_df, kick_int, dash)

In [125]:
replace_dash(kicking_df, kick_float, dash)

In [126]:
update_int_dtype(kicking_df, kick_int)

In [127]:
adjust_float(kicking_df, kick_float)

In [128]:
update_float_dtype(kicking_df, kick_float)

In [129]:
kicking_df.head()

,PLAYER,POSITION,TEAM,GP,FGM_A,FG%,LNG,_1_19,_20_29,_30_39,_40_49,OVER50,XPM_A,XP%,PTS,TTL,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7
0,Randy Bullock,K,CIN,7,19,89.5,55,0-0,3-3,6-7,6-7,2-2,16,100.0,67,77.0,10.0,13.0,12.0,17.0,3.0,12.0,10.0
1,Joey Slye,K,CAR,7,20,85.0,48,0-0,8-8,5-5,4-5,0-2,15,86.7,64,68.0,12.0,5.0,15.0,7.0,11.0,11.0,7.0
2,Rodrigo Blankenship,K,IND,6,18,88.9,44,0-0,4-4,7-8,5-6,0-0,15,100.0,63,68.0,8.0,15.0,12.0,14.0,11.0,8.0,0.0
3,Justin Tucker,K,BAL,6,14,92.9,55,0-0,2-2,3-3,7-7,1-2,20,100.0,59,68.0,9.0,16.0,9.0,8.0,10.0,16.0,0.0
4,Harrison Butker,K,KC,7,14,92.9,58,1-1,3-3,6-6,1-2,2-2,24,79.2,58,63.0,10.0,13.0,4.0,8.0,6.0,8.0,14.0


In [130]:
kicking_df.dtypes

PLAYER       object
POSITION     object
TEAM         object
GP            int64
FGM_A         int64
FG%         float64
LNG           int64
_1_19        object
_20_29       object
_30_39       object
_40_49       object
OVER50       object
XPM_A         int64
XP%         float64
PTS           int64
TTL         float64
Week_1      float64
Week_2      float64
Week_3      float64
Week_4      float64
Week_5      float64
Week_6      float64
Week_7      float64
dtype: object

## Pickle Cleaned DataFrames

In [131]:
offense.to_pickle('players')
team_names.to_pickle('long_names')
defense_df.to_pickle('defense')
kicking_df.to_pickle('kicking')